In [13]:
import numpy as np
# the layer number of each stage
stage_layer_max = [2, 4, 14, 2]
# the channel number of each layer
layer = np.array([64, 128, 256, 512])
# the descending step of channel 
step = 8 
op = ['repvgg','vgg'] # every stage
maxpool = [True,False]
ratio = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]
def search_space(stage_layer, layer):
    sum = 1
    for i in stage_layer_max:
        sum *= i
    for j in layer:
        sum *= len(ratio)
    return sum*16*16
print(search_space(stage_layer_max,layer)/1e8)

2.34881024


In [ ]:
# 训练自动生成yaml
# loss acc train val
# 存test最好的
# loss crossentropy
# SGD
# lr 0.1，0.001（repvgg
# weight decay 0.005，0.05（repvgg
# 直接量化repvgg， repvgg block 的量化，需要考虑lr和weight decay

# 随机生成模型配置，训
# 得到小数据集
# acc predictor， 是配置 （nas + predictor
# 搜索算法 RL EL （chip memory 约束项，要提acc，降latency
# 硬件性能 
# hardware performance 
# 硬件结构固定，公式计算硬件性能
# few-shot learning
# meta-learning
# 搜索考虑memory

# 1. search space
# 2. 重点是predictor
# 3. 

# random genea
# 进一步， 搜索对不同硬件都高效的网络
# 



In [7]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
import numpy as np

/home/qhy/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import numpy as np
import copy
from random import choices 
def repvgg_model_convert(model, do_copy=True):
    if do_copy:
        deploy_model = copy.deepcopy(model)
    for module in deploy_model.modules():
        if hasattr(module, 'switch_to_deploy'):
            module.switch_to_deploy()
    return deploy_model
# the layer number of each stage
stage_layer_max = [2, 4, 6, 8, 4]
# the channel number of each layer
layer_num_max = [64, 64, 128, 256, 512]
# the descending step of channel 
step = 8 
op = ['vgg','repvgg'] # every stage
pool_types = [True,False]
ratio = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]
datasets = ["cifar100"]
pool_type = ["maxpool","avgpool"]
# operation 
# dataset = choices(datasets)[0]
# class_num = 10 if dataset == "cifar10" else 100
import yaml
import torch
import os 
import collections
import sys
sys.path.append("..")
from thop import profile 
from models import model as M
def save_dict_to_yaml(dict_value: dict, save_path: str):
    """dict保存为yaml"""
    with open(save_path, 'w') as file:
        file.write(yaml.dump(dict_value, allow_unicode=True))
 
def read_yaml_to_dict(yaml_path: str):
    with open(yaml_path) as file:
        dict_value = yaml.load(file.read(), Loader=yaml.FullLoader)
        return dict_value
savedirs = []
md_fmts = []
for n in range(110,120):
    config = {}
    pool = []
    dataset = choices(datasets)[0]
    # 随机生成模型
    stage_layer = [choices(range(1,stage_layer_max[0]+1))[0],choices(range(1,stage_layer_max[1]+1))[0],choices(range(1,stage_layer_max[2]+1))[0],choices(range(1,stage_layer_max[3]+1))[0],choices(range(1,stage_layer_max[4]+1))[0]]
    # stage_layer
    ratio_1 = choices(ratio)[0]
    stage_ratio = [ratio_1,ratio_1,choices(ratio)[0],choices(ratio)[0],choices(ratio)[0]]
    # stage_ratio
    with_pool = [False,choices(pool_types)[0],choices(pool_types)[0],choices(pool_types)[0],choices(pool_types)[0],choices(pool_types)[0]]
    # with_pool
    op_type = [choices(op)[0],choices(op)[0],choices(op)[0],choices(op)[0],choices(op)[0]]
    # with_last_pool = choices(pool_types)[0]
    
        
    for pool_or_not in with_pool:
        if pool_or_not:
             pool.append(choices(pool_type)[0])
        else:
            pool.append(None)
    # if with_last_pool:
    #     pool.append(choices(pool_type)[0])
    # else:
    #     pool.append(None)
    dir = str(n)+'-stage-' + str(stage_layer[0]) + '_' + str(stage_layer[1]) + '_' + str(stage_layer[2]) + '_' + str(stage_layer[3]) + '_' + str(stage_layer[4]) + '-ratio-' \
    + str(stage_ratio[0]) + '_' + str(stage_ratio[1]) + '_' + str(stage_ratio[2]) + '_' + str(stage_ratio[3]) + '_' + str(stage_ratio[4]) +  \
        '-op-' + str(op_type[0]) + '_' + str(op_type[1]) + '_' + str(op_type[2]) + '_' + str(op_type[3]) + '_' + str(op_type[4]) + '-pool-' + \
            str(with_pool[0]) + '_' + str(with_pool[1]) + '_' + str(with_pool[2]) + '_' + str(with_pool[3]) + '_' + str(with_pool[4]) + '_' + str(with_pool[5]) + '-pool_type-'+\
                str(pool[0]) + '_' + str(pool[1]) + '_' + str(pool[2]) + '_' + str(pool[3]) + '_' + str(pool[4]) + '_' + str(pool[5]) + '-' + dataset
    
    config['dataset'] = dataset
    config['model'] = {'stage_layer':stage_layer,'stage_ratio': stage_ratio ,'with_pool':with_pool,"pool_type":pool,'op_type':op_type,"layer_num_max":layer_num_max}
    config['train'] = {'start_epoch': 0, "epochs": 100, "warmup_epochs": 0, "warmup_lr": 1.0e-2, 
                    "lr_scheduler":{"min_lr":1.0e-5,"decay_epochs":30,"decay_rate":0.1},"batch_size":256,"loss":"crossentropy","workers":4,
                    "optimizer":{"name":"SGD","base_lr":0.5,"repvgg_lr":0.5,"momentum":0.9,"eps":1.0e-8,"betas":[0.9,0.999],"weight_decay_param":{"base_decay":5e-4,"repvgg_decay":1e-4,"echo":False}},
                    "label_smoothing":0.1,"clip_grad":0.0,"ema_alpha":0.0,"ema_update_period":8,"use_l2_norm":True,
                    "no_weight_decay": ["rbr_dense","rbr_1x1"]}
    config['val'] = {"batch_size":128,"workers":4} 
    if dataset == "cifar100":
        save_path = os.path.join("../config/cifar100",dir+'.yaml')
        config['output'] = {"print_freq":100,"epoch_print_freq":1,"save_freq":20,
                        "dir": os.path.join("./log/cifar100",dir), "name": str(n)}
    else:
        save_path = os.path.join("../config/cifar10",dir+'.yaml')
        config['output'] = {"print_freq":100,"epoch_print_freq":1,"save_freq":20,
                        "dir": os.path.join("./log/cifar10",dir), "name": str(n)}
    model = M.Net(config, 10)
    model = repvgg_model_convert(model)
    input = torch.randn(1, 3, 32, 32)
    flops,params = profile(model, inputs=(input, ))
    md_fmt = "|"+ str(n)+'|' + str(stage_layer[0]) + '-' + str(stage_layer[1]) + '-' + str(stage_layer[2]) + '-' + str(stage_layer[3]) + '-' + str(stage_layer[4]) + '|'\
        + str(stage_ratio[0]) + '-' + str(stage_ratio[1]) + '-' + str(stage_ratio[2]) + '-' + str(stage_ratio[3]) + '-' + str(stage_ratio[4]) + \
            '|' + str(op_type[0]) + '-' + str(op_type[1]) + '-' + str(op_type[2]) + '-' + str(op_type[3]) + '-' + str(op_type[4]) + '|' + \
                str(with_pool[0]) + '-' + str(with_pool[1]) + '-' + str(with_pool[2]) + '-' + str(with_pool[3]) + '-' + str(with_pool[4]) + '|' +\
                str(pool[0]) + '-' + str(pool[1]) + '-' + str(pool[2]) + '-' + str(pool[3]) + '-' + str(pool[4]) + '-' + str(pool[5]) + '|' + dataset+ "|" + str(round(flops/1e6,2))+"|"+str(round(params/1e6,2))+"|"+"|"
    md_fmts.append(md_fmt)
    savedirs.append(save_path.replace("..","."))
    save_dict_to_yaml(config,save_path)

print("="*100)
for md in md_fmts:
    print(md)
print("="*100)
for savedir in savedirs:
    print("python3 train.py --config " + savedir)

/home/qhy/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for

In [10]:
from easydict import EasyDict
import yaml
sys.path.append("..")
from thop import profile 
from models import model as M
config_path = "/home/qhy/Reserach/AICAS/config/914-stage-1_3_2_5_1-ratio-1.0_1.0_0.25_0.375_0.75-op-vgg_vgg_vgg_repvgg_repvgg-pool-False_True_False_True_True_False-pool_type-None_avgpool_None_avgpool_maxpool_None-cifar10.yaml"
config = EasyDict(yaml.full_load(open(config_path)))
model = M.Net(config, 10)
# model